In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [3]:
!pip install transformers

In [4]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import T5Tokenizer, T5EncoderModel
from sklearn.decomposition import PCA
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including T5 embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize T5 model and tokenizer
    model_name = "t5-small"  # Change to a larger T5 model if required
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5EncoderModel.from_pretrained(model_name)

    # Generate T5 embeddings for the text column
    def encode_text(texts):
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    embeddings = np.vstack([encode_text([text]) for text in df[text_column].tolist()])

    # Convert embeddings to DataFrame
    t5_df = pd.DataFrame(embeddings, columns=[f't5_{i}' for i in range(embeddings.shape[1])])

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], t5_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'confidence'
y = df['confidence'].apply(lambda x: x + 1)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 10}
CatBoost Model Metrics:
Train Accuracy: 0.9614
Train Precision: 0.9627
Train Recall: 0.9614
Train F1 Score: 0.9607
Train Confusion Matrix:
[[ 27   1   4]
 [  0 131  15]
 [  0   1 365]]

Test Accuracy: 0.6350
Test Precision: 0.6070
Test Recall: 0.6350
Test F1 Score: 0.6037
Test Confusion Matrix:
[[ 1  2 10]
 [ 0  6 21]
 [ 1 16 80]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6765
Train Precision: 0.7227
Train Recall: 0.6765
Train F1 Score: 0.5496
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 144]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]



In [12]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import T5Tokenizer, T5EncoderModel
from sklearn.decomposition import PCA
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including T5 embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize T5 model and tokenizer
    model_name = "t5-small"  # Change to a larger T5 model if required
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5EncoderModel.from_pretrained(model_name)

    # Generate T5 embeddings for the text column
    def encode_text(texts):
        # Ensure 'texts' is a string or a list of strings
        if not isinstance(texts, (str, list)):
            # If not a string or list, convert it to a string
            texts = str(texts)
        if isinstance(texts, str):
            texts = [texts]  # If it's a single string, wrap it in a list
        elif not all(isinstance(item, str) for item in texts):
            # If any item in the list is not a string, convert it to a string
            texts = [str(item) for item in texts]
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    embeddings = np.vstack([encode_text(text) for text in df[text_column].tolist()])

    # Convert embeddings to DataFrame
    # The indentation of this line was corrected
    t5_df = pd.DataFrame(embeddings, columns=[f't5_{i}' for i in range(embeddings.shape[1])])

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], t5_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'confidence'
y = df['confidence'].apply(lambda x: x + 1)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 4
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 11}
KNN Model Metrics:
Train Accuracy: 0.9926
Train Precision: 0.9926
Train Recall: 0.9926
Train F1 Score: 0.9926
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   2 364]]

Test Accuracy: 0.6496
Test Precision: 0.5602
Test Recall: 0.6496
Test F1 Score: 0.5987
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  5 22]
 [ 1 12 84]]

Running Random Forest...
Best Params: {'n_estimators': 150, 'min_samples_split': 2, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9926
Train Precision: 0.9927
Train Recall: 0.9926
Train F1 Score: 0.9926
Train Confusion Matrix:
[[ 32   0   0]
 [  0 142   4]
 [  0   0 366]]

Test Accuracy: 0.6496
Test Precision: 0.5693
Test Recall: 0.6496
Test F1 Score: 0.6055
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  7 20]
 [ 0 15 82

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.6949
Train Precision: 0.6937
Train Recall: 0.6949
Train F1 Score: 0.6098
Train Confusion Matrix:
[[  1   4  27]
 [  0  19 127]
 [  0   8 358]]

Test Accuracy: 0.6788
Test Precision: 0.5292
Test Recall: 0.6788
Test F1 Score: 0.5855
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  1 26]
 [ 0  5 92]]

Running XGBoost...
Best Params: {'subsample': 0.8, 'n_estimators': 50, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9393
Train Precision: 0.9425
Train Recall: 0.9393
Train F1 Score: 0.9375
Train Confusion Matrix:
[[ 24   2   6]
 [  0 122  24]
 [  0   1 365]]

Test Accuracy: 0.6058
Test Precision: 0.5440
Test Recall: 0.6058
Test F1 Score: 0.5731
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  6 21]
 [ 1 19 77]]

Running AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running CatBoost...


...

Best Params: {'learning_rate': 0.1, 'iterations': 150, 'depth': 10}
CatBoost Model Metrics:
Train Accuracy: 0.9577
Train Precision: 0.9598
Train Recall: 0.9577
Train F1 Score: 0.9568
Train Confusion Matrix:
[[ 27   1   4]
 [  0 128  18]
 [  0   0 366]]

Test Accuracy: 0.6569
Test Precision: 0.5675
Test Recall: 0.6569
Test F1 Score: 0.6062
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  6 21]
 [ 0 13 84]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6783
Train Precision: 0.7235
Train Recall: 0.6783
Train F1 Score: 0.5538
Train Confusion Matrix:
[[  0   0  32]
 [  0   3 143]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]



In [13]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import T5Tokenizer, T5EncoderModel
from sklearn.decomposition import PCA
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including T5 embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize T5 model and tokenizer
    model_name = "t5-small"  # Change to a larger T5 model if required
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5EncoderModel.from_pretrained(model_name)

    # Generate T5 embeddings for the text column
    def encode_text(texts):
        # Ensure 'texts' is a string or a list of strings
        if not isinstance(texts, (str, list)):
            # If not a string or list, convert it to a string
            texts = str(texts)
        if isinstance(texts, str):
            texts = [texts]  # If it's a single string, wrap it in a list
        elif not all(isinstance(item, str) for item in texts):
            # If any item in the list is not a string, convert it to a string
            texts = [str(item) for item in texts]
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    embeddings = np.vstack([encode_text(text) for text in df[text_column].tolist()])

    # Convert embeddings to DataFrame
    # The indentation of this line was corrected
    t5_df = pd.DataFrame(embeddings, columns=[f't5_{i}' for i in range(embeddings.shape[1])])

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], t5_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'


# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'confidence'
y = df['confidence'].apply(lambda x: x + 1)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 4
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 11}
KNN Model Metrics:
Train Accuracy: 0.9926
Train Precision: 0.9926
Train Recall: 0.9926
Train F1 Score: 0.9926
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   2 364]]

Test Accuracy: 0.6496
Test Precision: 0.5602
Test Recall: 0.6496
Test F1 Score: 0.5987
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  5 22]
 [ 1 12 84]]

Running Random Forest...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9926
Train Precision: 0.9927
Train Recall: 0.9926
Train F1 Score: 0.9926
Train Confusion Matrix:
[[ 32   0   0]
 [  0 142   4]
 [  0   0 366]]

Test Accuracy: 0.6204
Test Precision: 0.5319
Test Recall: 0.6204
Test F1 Score: 0.5713
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  4 23]
 [ 0 16 81]]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.6949
Train Precision: 0.6542
Train Recall: 0.6949
Train F1 Score: 0.6000
Train Confusion Matrix:
[[  0   3  29]
 [  0  16 130]
 [  0   4 362]]

Test Accuracy: 0.6861
Test Precision: 0.5355
Test Recall: 0.6861
Test F1 Score: 0.5895
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  1 26]
 [ 0  4 93]]

Running XGBoost...
Best Params: {'subsample': 0.8, 'n_estimators': 50, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9393
Train Precision: 0.9420
Train Recall: 0.9393
Train F1 Score: 0.9374
Train Confusion Matrix:
[[ 24   3   5]
 [  0 122  24]
 [  0   1 365]]

Test Accuracy: 0.6204
Test Precision: 0.5441
Test Recall: 0.6204
Test F1 Score: 0.5789
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  5 22]
 [ 1 16 80]]

Running AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running CatBoost...


...

Best Params: {'learning_rate': 0.1, 'iterations': 100, 'depth': 10}
CatBoost Model Metrics:
Train Accuracy: 0.9246
Train Precision: 0.9301
Train Recall: 0.9246
Train F1 Score: 0.9203
Train Confusion Matrix:
[[ 18   3  11]
 [  0 119  27]
 [  0   0 366]]

Test Accuracy: 0.6204
Test Precision: 0.5319
Test Recall: 0.6204
Test F1 Score: 0.5713
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  4 23]
 [ 0 16 81]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6783
Train Precision: 0.7235
Train Recall: 0.6783
Train F1 Score: 0.5538
Train Confusion Matrix:
[[  0   0  32]
 [  0   3 143]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]



In [14]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import T5Tokenizer, T5EncoderModel
from sklearn.decomposition import PCA
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including T5 embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize T5 model and tokenizer
    model_name = "t5-small"  # Change to a larger T5 model if required
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5EncoderModel.from_pretrained(model_name)

    # Generate T5 embeddings for the text column
    def encode_text(texts):
        # Ensure 'texts' is a string or a list of strings
        if not isinstance(texts, (str, list)):
            # If not a string or list, convert it to a string
            texts = str(texts)
        if isinstance(texts, str):
            texts = [texts]  # If it's a single string, wrap it in a list
        elif not all(isinstance(item, str) for item in texts):
            # If any item in the list is not a string, convert it to a string
            texts = [str(item) for item in texts]
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    embeddings = np.vstack([encode_text(text) for text in df[text_column].tolist()])

    # Convert embeddings to DataFrame
    # The indentation of this line was corrected
    t5_df = pd.DataFrame(embeddings, columns=[f't5_{i}' for i in range(embeddings.shape[1])])

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], t5_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'confidence'
y = df['confidence'].apply(lambda x: x + 1)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 2
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 8}
KNN Model Metrics:
Train Accuracy: 0.9945
Train Precision: 0.9945
Train Recall: 0.9945
Train F1 Score: 0.9945
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   1 365]]

Test Accuracy: 0.6350
Test Precision: 0.6258
Test Recall: 0.6350
Test F1 Score: 0.6147
Test Confusion Matrix:
[[ 1  2 10]
 [ 0  9 18]
 [ 1 19 77]]

Running Random Forest...
Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9522
Train Precision: 0.9547
Train Recall: 0.9522
Train F1 Score: 0.9483
Train Confusion Matrix:
[[ 17   2  13]
 [  0 135  11]
 [  0   0 366]]

Test Accuracy: 0.6277
Test Precision: 0.6417
Test Recall: 0.6277
Test F1 Score: 0.5942
Test Confusion Matrix:
[[ 1  2 10]
 [ 0  5 22]
 [ 0 17 80]]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running XGBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 0.8, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9908
Train Precision: 0.9909
Train Recall: 0.9908
Train F1 Score: 0.9908
Train Confusion Matrix:
[[ 32   0   0]
 [  0 141   5]
 [  0   0 366]]

Test Accuracy: 0.5985
Test Precision: 0.6480
Test Recall: 0.5985
Test F1 Score: 0.5847
Test Confusion Matrix:
[[ 1  2 10]
 [ 0  8 19]
 [ 0 24 73]]

Running AdaBoost...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 10}
CatBoost Model Metrics:
Train Accuracy: 0.9614
Train Precision: 0.9627
Train Recall: 0.9614
Train F1 Score: 0.9607
Train Confusion Matrix:
[[ 27   1   4]
 [  0 131  15]
 [  0   1 365]]

Test Accuracy: 0.6350
Test Precision: 0.6070
Test Recall: 0.6350
Test F1 Score: 0.6037
Test Confusion Matrix:
[[ 1  2 10]
 [ 0  6 21]
 [ 1 16 80]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6765
Train Precision: 0.7227
Train Recall: 0.6765
Train F1 Score: 0.5496
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 144]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

